## 算法记忆版

In [1]:
import numpy as np  # 导入numpy库，用于科学计算

# 定义一些全局变量
DNA_SIZE = 24  # DNA的长度，这里是24位二进制数
POP_SIZE = 200  # 种群的大小，这里是200个个体
CROSSOVER_RATE = 0.8  # 交叉的概率，这里是80%
MUTATION_RATE = 0.005  # 变异的概率，这里是0.5%
N_GENERATIONS = 50  # 迭代的次数，这里是50次
X_BOUND = [-3, 3]  # x的取值范围，这里是[-3, 3]

def F(x): 
    # 定义适应度函数，这里是一个复杂的数学函数
    return np.sin(10*x)*x + np.cos(2*x)*x     

def get_fitness(pred): 
    # 定义计算适应度的函数，适应度越高，个体越有可能被选中
    return pred + 1e-3 - np.min(pred)          

def translateDNA(pop): 
    # 定义将二进制DNA转换为十进制的x的函数
    return pop.dot(2**np.arange(DNA_SIZE)[::-1]) / float(2**DNA_SIZE-1) * (X_BOUND[1] - X_BOUND[0]) + X_BOUND[0]

def crossover_and_mutation(pop):
    # 定义交叉和变异的函数
    new_pop = []  # 创建一个新的种群
    for parent in pop:  # 对于种群中的每一个个体
        child = parent  # 孩子初始与父母相同
        if np.random.rand() < CROSSOVER_RATE:  # 如果随机数小于交叉概率
            other = pop[np.random.randint(POP_SIZE)]  # 随机选择另一个个体
            cross_points = np.random.randint(low=0, high=DNA_SIZE)  # 随机选择交叉点
            child[cross_points:] = other[cross_points:]  # 交叉
        mutation(child)  # 变异
        new_pop.append(child)  # 将孩子添加到新的种群中
    return new_pop  # 返回新的种群

def mutation(child):
    # 定义变异的函数
    if np.random.rand() < MUTATION_RATE:  # 如果随机数小于变异概率
        mutate_point = np.random.randint(0, DNA_SIZE)  # 随机选择一个点进行变异
        child[mutate_point] = child[mutate_point]^1  # 变异

def select(pop, fitness):
    # 定义选择的函数，适应度越高的个体被选择的概率越大
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return pop[idx]  # 返回选择后的种群

pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE))  # 初始化种群，每个个体的DNA是24位的二进制数

for _ in range(N_GENERATIONS):  # 进行50次迭代
    F_values = F(translateDNA(pop))  # 计算种群中每个个体的适应度
    fitness = get_fitness(F_values)  # 计算种群中每个个体的适应度值
    pop = select(pop, fitness)  # 选择
    pop = np.array(crossover_and_mutation(pop))  # 交叉和变异

print("最优解：", translateDNA(pop[np.argmax(fitness)]))  # 输出最优解
print("最大适应度：", F(translateDNA(pop[np.argmax(fitness)])))  # 输出最大适应度

最优解： -2.0391132258840337
最大适应度： 3.2464236531492907


In [9]:
import numpy as np
import random

# 参数设置
min_range = -513  # 种群中个体的最小值
max_range = 513  # 种群中个体的最大值
dimension = 2  # 种群的维度
factor = 0.8  # 缩放因子
rounds = 100  # 进化的轮数
size = 100  # 种群的大小
CR = 0.75  # 交叉概率

# 目标函数
def f(v):
    return -(v[1]+47)*np.sin(np.sqrt(np.abs(v[1]+(v[0]/2)+47))) - v[0] * np.sin(np.sqrt(np.abs(v[0]-v[1]-47)))

# 初始化种群
def initialize_population(size, dimension, min_range, max_range):
    return [np.array([random.uniform(min_range, max_range) for _ in range(dimension)]) for _ in range(size)]

# 变异操作
def mutate(individual, size, factor):
    r0, r1, r2 = np.random.choice(size, 3, replace=False)
    return individual[r0] + factor * (individual[r1] - individual[r2])

# 交叉操作
def crossover(mutant, target, CR):
    mask = np.random.rand(len(mutant)) < CR
    return np.where(mask, mutant, target)

# 进化操作
def evolve(individuality, size, dimension, factor, CR, f):
    for i in range(size):
        mutant = mutate(individuality, size, factor)
        trial = crossover(mutant, individuality[i], CR)
        if f(trial) < f(individuality[i]):
            individuality[i] = trial
    return individuality

# 主函数
def main():
    individuality = initialize_population(size, dimension, min_range, max_range)
    for _ in range(rounds):
        individuality = evolve(individuality, size, dimension, factor, CR, f)
    # 找出最优解
    best_individual = min(individuality, key=f)
    print("最优解：", best_individual)
    print("目标函数值：", f(best_individual))


main()

最优解： [ 2.05256268e+09 -7.06860668e+15]
目标函数值： -5137903427515349.0


In [7]:
import numpy as np

# 参数设置
bounds = [(-513, 513), (-513, 513)]
particle_count = 100
iterations = 100

# 目标函数
def f(v):
    return -(v[1]+47)*np.sin(np.sqrt(np.abs(v[1]+(v[0]/2)+47))) - v[0] * np.sin(np.sqrt(np.abs(v[0]-v[1]-47)))

# 初始化粒子
def initialize_particle(bounds):
    position = np.random.uniform(bounds[0], bounds[1], len(bounds))
    velocity = np.random.uniform(-1, 1, len(bounds))
    return position, velocity

# 更新粒子速度
def update_velocity(position, velocity, best_position, global_best):
    r1, r2 = np.random.rand(2, len(velocity))
    cognitive_velocity = 1.4 * r1 * (best_position - position)
    social_velocity = 1.4 * r2 * (global_best - position)
    return 0.7 * velocity + cognitive_velocity + social_velocity

# 更新粒子位置
def update_position(position, velocity, bounds):
    position += velocity
    return np.clip(position, bounds[0], bounds[1])

# 更新全局最优
def update_global_best(best_positions, global_best, f):
    current_best = max(best_positions, key=f)
    if f(current_best) > f(global_best):
        global_best = current_best
    return global_best

# 粒子群优化
def particle_swarm_optimization(f, particle_count, bounds, iterations):
    particles = [initialize_particle(bounds) for _ in range(particle_count)]
    best_positions = [p[0] for p in particles]
    global_best = max(best_positions, key=f)

    for _ in range(iterations):
        for i in range(particle_count):
            position, velocity = particles[i]
            velocity = update_velocity(position, velocity, best_positions[i], global_best)
            position = update_position(position, velocity, bounds)
            particles[i] = (position, velocity)
            if f(position) > f(best_positions[i]):
                best_positions[i] = position
        global_best = update_global_best(best_positions, global_best, f)

    return global_best

# 主函数
def main():
    result = particle_swarm_optimization(f, particle_count, bounds, iterations)
    print("最优解：", result)
    print("目标函数值：", f(result))

main()

最优解： [-514.59264913  512.27502418]
目标函数值： 1059.0951232098614
